## Exercise 3

Defining functions used:

In [1]:
from typing import Union, Any
import pandas as pd

def get_filename(month: str, directory: str = 'dataset') -> str:
    PREFIX = 'Sales'
    YEAR = '2019'
    return f"{directory}/{PREFIX}_{month}_{YEAR}.csv"

def get_csv_files(directory: str) -> pd.DataFrame:
    MONTHS = [
        'January', 
        'February', 
        'March', 
        'April',  
        'May', 
        'June', 
        'July', 
        'August', 
        'September',
        'October', 
        'November', 
        'December'
    ]

    csv_files = []
    for month in MONTHS:
        csv_filename = get_filename(month, directory)
        csv_file = pd.read_csv(csv_filename, index_col=0)
        csv_files.append(csv_file)

    return pd.concat(csv_files, ignore_index=True).reset_index(drop=True)

def remove_noise_columns(csv_file: pd.DataFrame, columns: list) -> pd.DataFrame:
    for column in columns:
        del csv_file[column]

    return csv_file.reset_index(drop=True)

# def get_month_number(csv_file: Union[Any, pd.DataFrame, pd.Series, pd.Index]) -> Union[Any, pd.DataFrame, pd.Series, pd.Index]:
def get_month_number(csv_file: Any) -> pd.DataFrame:
    dates = csv_file['Order Date']
    return dates.astype('str').str[0:2]

def add_column(csv_file: Union[pd.DataFrame, pd.Series], column: str, values: Any) -> Union[pd.DataFrame, pd.Series]:
    csv_file.loc[:, column] = values
    return csv_file

def convert_to_numeric(csv_file: Any, column: str) -> pd.Series:
    csv_file.loc[:, column] = pd.to_numeric(csv_file[column], errors='coerce')
    return csv_file[pd.to_numeric(csv_file[column], errors='coerce').notnull()]

def write_csv(csv_file: Union[pd.DataFrame, pd.Series], csv_filename: str) -> Union[None, str]:
    csv_file.reset_index(drop=True).to_csv(csv_filename)

1. Reading and combining csv files into a single DataFrama:

In [2]:
csv_file1 = get_csv_files('data')
display(csv_file1.tail())

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,x_t,perf
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",50,25
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",22,27
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",6,36
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001",30,46
186849,319670,Bose SoundSport Headphones,1,99.99,12/21/19 21:45,"747 Chestnut St, Los Angeles, CA 90001",34,41


2. Removing noise columns:

In [3]:
csv_file2 = remove_noise_columns(csv_file1, ['x_t', 'perf']).reset_index(drop=True)
display(csv_file2.tail())

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016"
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001"
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101"
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001"
186849,319670,Bose SoundSport Headphones,1,99.99,12/21/19 21:45,"747 Chestnut St, Los Angeles, CA 90001"


3a, b, c. Adding month numbers and converting to numeric:

In [4]:
month_numbers = get_month_number(csv_file2)
csv_file3b = add_column(csv_file2, 'Month', month_numbers)
csv_file3c = convert_to_numeric(csv_file3b, 'Month')
display(csv_file3c.tail())

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",12.0
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",12.0
186847,319668,Vareebadd Phone,1,400,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",12.0
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001",12.0
186849,319670,Bose SoundSport Headphones,1,99.99,12/21/19 21:45,"747 Chestnut St, Los Angeles, CA 90001",12.0


3d. Adding sum column:

In [5]:
csv_file3d = convert_to_numeric(csv_file3c, 'Quantity Ordered')
csv_file3d = convert_to_numeric(csv_file3d, 'Price Each')
sum = csv_file3d['Quantity Ordered'].mul(csv_file3d['Price Each'])
csv_file3d = add_column(csv_file3d, 'Sum', sum)
display(csv_file3d.tail())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sum
186845,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",12.0,14.95
186846,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",12.0,7.68
186847,319668,Vareebadd Phone,1,400.00,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",12.0,400.00
186848,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001",12.0,11.99
186849,319670,Bose SoundSport Headphones,1,99.99,12/21/19 21:45,"747 Chestnut St, Los Angeles, CA 90001",12.0,99.99


3e. Grouping by month:

In [6]:
csv_file3e = csv_file3d.groupby(['Month']).sum().sort_values('Sum', ascending=False)
remove_noise_columns(csv_file3e, ['Price Each'])
display(csv_file3e) 

,Quantity Ordered,Sum
Month,,
12.0,28114,4613443.34
10.0,22703,3736726.88
4.0,20558,3390670.24
11.0,19798,3199603.20
5.0,18667,3152606.75
3.0,17005,2807100.38
7.0,16072,2647775.76
6.0,15253,2577802.26
8.0,13448,2244467.88


3d. Printing result to file:

In [7]:
write_csv(csv_file3e, 'result/oppgave_f.csv')